In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import seaborn as sns
import re

from scipy import stats
from scipy.stats import skew
from sklearn import linear_model
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LogisticRegression

import sklearn.model_selection as ms
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

pd.set_option('display.max_columns',99)
pd.set_option('display.max_rows',300)

In [2]:
test_dataset = pd.read_csv('./test_dataset.csv')
train_dataset = pd.read_csv('./train_dataset.csv')
#import datasets

In [3]:
print(train_dataset.shape)
print(test_dataset.shape)
#get shape of train and test data set

(91589, 50)
(10177, 50)


In [4]:
train_dataset['dataset'] = 'train'
test_dataset['dataset'] = 'test'
#add new column to make separating easier later

In [5]:
merged_data = pd.concat([train_dataset, test_dataset], axis=0)
merged_data.shape
#merge data set

(101766, 51)

In [6]:
df = merged_data.copy(deep=True)
#make a copy for data exploration

In [8]:
df.sample(5)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,dataset
75244,213143850,60500736,AfricanAmerican,Male,[80-90),?,1,1,7,2,CM,?,12,0,7,0,0,0,276,585,250.8,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,<30,train
89962,45447096,19471599,Hispanic,Male,[60-70),?,1,18,7,1,?,Family/GeneralPractice,51,0,5,0,0,0,786,401,250,6,None,>7,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30,train
34129,162903828,40403583,Caucasian,Male,[70-80),?,1,6,7,3,MC,?,72,1,11,1,0,0,532,185,197,9,None,None,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,train
82387,87949686,6783318,AfricanAmerican,Female,[60-70),?,2,1,1,4,MC,Orthopedics-Reconstructive,3,1,17,0,0,0,715,250,401,5,None,None,Steady,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,train
50299,374356352,85328064,Caucasian,Male,[70-80),?,1,6,7,2,MC,Emergency/Trauma,58,0,14,2,0,2,250.6,250.42,276,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,train


### Data Exploration

#### Exploring the target variable, readmitted rate

In [9]:
def summarize_feature(dataframe,feature):
    feature_summary = {'values':dataframe[feature].value_counts().index.tolist(),
                     'counts':dataframe[feature].value_counts().values.tolist(),
                     'frequency': dataframe[feature].value_counts(normalize = True).values.tolist()}
    feature_summary = pd.DataFrame(feature_summary)
    return feature_summary


In [10]:
summarize_feature(df, 'readmitted')

,values,counts,frequency
0,NO,54864,0.539119
1,>30,35545,0.349282
2,<30,11357,0.111599


In [11]:
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('NO', 0)
#replace target variable with numerical value


In [12]:
summarize_feature(df, 'readmitted')

,values,counts,frequency
0,0,90409,0.888401
1,1,11357,0.111599


### feature examination

In [13]:
summarize_feature(df, 'race')

,values,counts,frequency
0,Caucasian,76099,0.747784
1,AfricanAmerican,19210,0.188766
2,?,2273,0.022336
3,Hispanic,2037,0.020017
4,Other,1506,0.014799
5,Asian,641,0.006299


In [14]:
dict_race = {'?': -999, 'Caucasian':1, 'AfricanAmerican':2,'Asian':3,'Other':4,'Hispanic':5}
df.race.replace(dict_race, inplace=True)
#impute race feature with nominal values, '?' is imputed with -999

In [15]:
summarize_feature(df, 'race')

,values,counts,frequency
0,1,76099,0.747784
1,2,19210,0.188766
2,-999,2273,0.022336
3,5,2037,0.020017
4,4,1506,0.014799
5,3,641,0.006299


In [16]:
summarize_feature(df, 'gender')

,values,counts,frequency
0,Female,54708,0.537586
1,Male,47055,0.462384
2,Unknown/Invalid,3,0.000029


In [17]:
df = df[df.gender != 'Unknown/Invalid']
#only 3 unknown/invalid, drop rows

In [18]:
dict_gender = {'Female':0, 'Male':1}
df.gender.replace(dict_gender, inplace=True)
#impute gender feature with nominal value


In [19]:
summarize_feature(df, 'gender')

,values,counts,frequency
0,0,54708,0.537602
1,1,47055,0.462398


In [20]:
summarize_feature(df, 'age')

,values,counts,frequency
0,[70-80),26066,0.256144
1,[60-70),22482,0.220925
2,[50-60),17256,0.169570
3,[80-90),17197,0.168991
4,[40-50),9685,0.095172
5,[30-40),3775,0.037096
6,[90-100),2793,0.027446
7,[20-30),1657,0.016283
8,[10-20),691,0.006790
9,[0-10),161,0.001582


In [21]:
dict_age = {'[60-70)': 65, 
            '[40-50)':45, 
            '[80-90)':85,
               '[70-80)':75,
               '[50-60)':55,
               '[20-30)':25,
               '[90-100)':95,
               '[30-40)':35,
               '[0-10)':5,
               '[10-20)':15}
df.age.replace(dict_age, inplace=True)
#impute age feature with median num


In [22]:
summarize_feature(df, 'age')

,values,counts,frequency
0,75,26066,0.256144
1,65,22482,0.220925
2,55,17256,0.169570
3,85,17197,0.168991
4,45,9685,0.095172
5,35,3775,0.037096
6,95,2793,0.027446
7,25,1657,0.016283
8,15,691,0.006790
9,5,161,0.001582


In [23]:
# Creating additional columns for diagnosis
df['level1_diag1'] = df['diag_1']
df['level1_diag2'] = df['diag_2']
df['level1_diag3'] = df['diag_3']


In [24]:
df.loc[df['diag_1'].str.contains('V'), ['level1_diag1']] = 0
df.loc[df['diag_1'].str.contains('E'), ['level1_diag1']] = 0
df.loc[df['diag_2'].str.contains('V'), ['level1_diag2']] = 0
df.loc[df['diag_2'].str.contains('E'), ['level1_diag2']] = 0
df.loc[df['diag_3'].str.contains('V'), ['level1_diag3']] = 0
df.loc[df['diag_3'].str.contains('E'), ['level1_diag3']] = 0
df['level1_diag1'] = df['level1_diag1'].replace('?', -1)
df['level1_diag2'] = df['level1_diag2'].replace('?', -1)
df['level1_diag3'] = df['level1_diag3'].replace('?', -1)


In [25]:
df['level1_diag1'] = df['level1_diag1'].astype(float)
df['level1_diag2'] = df['level1_diag2'].astype(float)
df['level1_diag3'] = df['level1_diag3'].astype(float)


In [26]:
for index, row in df.iterrows():
    if (row['level1_diag1'] >= 390 and row['level1_diag1'] < 460) or (np.floor(row['level1_diag1']) == 785):
        df.loc[index, 'level1_diag1'] = 1
    elif (row['level1_diag1'] >= 460 and row['level1_diag1'] < 520) or (np.floor(row['level1_diag1']) == 786):
        df.loc[index, 'level1_diag1'] = 2
    elif (row['level1_diag1'] >= 520 and row['level1_diag1'] < 580) or (np.floor(row['level1_diag1']) == 787):
        df.loc[index, 'level1_diag1'] = 3
    elif (np.floor(row['level1_diag1']) == 250):
        df.loc[index, 'level1_diag1'] = 4
    elif (row['level1_diag1'] >= 800 and row['level1_diag1'] < 1000):
        df.loc[index, 'level1_diag1'] = 5
    elif (row['level1_diag1'] >= 710 and row['level1_diag1'] < 740):
        df.loc[index, 'level1_diag1'] = 6
    elif (row['level1_diag1'] >= 580 and row['level1_diag1'] < 630) or (np.floor(row['level1_diag1']) == 788):
        df.loc[index, 'level1_diag1'] = 7
    elif (row['level1_diag1'] >= 140 and row['level1_diag1'] < 240):
        df.loc[index, 'level1_diag1'] = 8
    else:
        df.loc[index, 'level1_diag1'] = 0
        
    if (row['level1_diag2'] >= 390 and row['level1_diag2'] < 460) or (np.floor(row['level1_diag2']) == 785):
        df.loc[index, 'level1_diag2'] = 1
    elif (row['level1_diag2'] >= 460 and row['level1_diag2'] < 520) or (np.floor(row['level1_diag2']) == 786):
        df.loc[index, 'level1_diag2'] = 2
    elif (row['level1_diag2'] >= 520 and row['level1_diag2'] < 580) or (np.floor(row['level1_diag2']) == 787):
        df.loc[index, 'level1_diag2'] = 3
    elif (np.floor(row['level1_diag2']) == 250):
        df.loc[index, 'level1_diag2'] = 4
    elif (row['level1_diag2'] >= 800 and row['level1_diag2'] < 1000):
        df.loc[index, 'level1_diag2'] = 5
    elif (row['level1_diag2'] >= 710 and row['level1_diag2'] < 740):
        df.loc[index, 'level1_diag2'] = 6
    elif (row['level1_diag2'] >= 580 and row['level1_diag2'] < 630) or (np.floor(row['level1_diag2']) == 788):
        df.loc[index, 'level1_diag2'] = 7
    elif (row['level1_diag2'] >= 140 and row['level1_diag2'] < 240):
        df.loc[index, 'level1_diag2'] = 8
    else:
        df.loc[index, 'level1_diag2'] = 0
    
    if (row['level1_diag3'] >= 390 and row['level1_diag3'] < 460) or (np.floor(row['level1_diag3']) == 785):
        df.loc[index, 'level1_diag3'] = 1
    elif (row['level1_diag3'] >= 460 and row['level1_diag3'] < 520) or (np.floor(row['level1_diag3']) == 786):
        df.loc[index, 'level1_diag3'] = 2
    elif (row['level1_diag3'] >= 520 and row['level1_diag3'] < 580) or (np.floor(row['level1_diag3']) == 787):
        df.loc[index, 'level1_diag3'] = 3
    elif (np.floor(row['level1_diag3']) == 250):
        df.loc[index, 'level1_diag3'] = 4
    elif (row['level1_diag3'] >= 800 and row['level1_diag3'] < 1000):
        df.loc[index, 'level1_diag3'] = 5
    elif (row['level1_diag3'] >= 710 and row['level1_diag3'] < 740):
        df.loc[index, 'level1_diag3'] = 6
    elif (row['level1_diag3'] >= 580 and row['level1_diag3'] < 630) or (np.floor(row['level1_diag3']) == 788):
        df.loc[index, 'level1_diag3'] = 7
    elif (row['level1_diag3'] >= 140 and row['level1_diag3'] < 240):
        df.loc[index, 'level1_diag3'] = 8
    else:
        df.loc[index, 'level1_diag3'] = 0

In [25]:
df.to_csv('./temp_merged_data_preprocessed.csv')
#above line takes time to process, making a temp csv

In [26]:
df = pd.read_csv('./temp_merged_data_preprocessed.csv')
#read the temp file back in

In [27]:
summarize_feature(df,'max_glu_serum')

,values,counts,frequency
0,None,96417,0.947466
1,Norm,2597,0.025520
2,>200,1485,0.014593
3,>300,1264,0.012421


In [28]:
dict_max_glu_seru = {'None': -999, 
                     'Norm':0, 
                     '>300':2,
                    '>200':1}
df.max_glu_serum.replace(dict_max_glu_seru, inplace=True)
#replace max_glu_serum with numerical values

In [29]:
summarize_feature(df,'max_glu_serum')
#check again

,values,counts,frequency
0,-999,96417,0.947466
1,0,2597,0.025520
2,1,1485,0.014593
3,2,1264,0.012421


In [30]:
summarize_feature(df,'A1Cresult')

,values,counts,frequency
0,None,84745,0.832768
1,>8,8216,0.080737
2,Norm,4990,0.049036
3,>7,3812,0.037460


In [31]:
dict_A1Cresult = {'None': -999, 
                     'Norm':0, 
                     '>7':1,
                    '>8':2}
df.A1Cresult.replace(dict_A1Cresult, inplace=True)
#replace A1C with numerical values

In [32]:
summarize_feature(df,'A1Cresult')
#check again

,values,counts,frequency
0,-999,84745,0.832768
1,2,8216,0.080737
2,0,4990,0.049036
3,1,3812,0.037460


In [33]:
#here we want to note any changes in the dosage of medicine.
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
for col in keys:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
df['med_dosage_change'] = 0
for col in keys:
    colname = str(col) + 'temp'
    df['med_dosage_change'] = df['med_dosage_change'] + df[colname]
    del df[colname]
#here 0 means no changes in doses of medicine, >0 means there is increase or decrease in dosage

In [34]:
df['number_of_medicine'] = 0
for col in keys:
    df['number_of_medicine'] = df['number_of_medicine'] + df[col].apply(lambda x: 0 if (x == 'No') else 1)
#new column to account for the total number of medicine patient is taking

In [35]:
drug_df = df.loc[:, 'metformin':'diabetesMed']
for drug in list(drug_df):
    print('\n'+drug)
    print('-'*50)
    n = summarize_feature(drug_df,  drug)
    print(n)
#values in drug list is has similar values, make 1 library


metformin
--------------------------------------------------
   values  counts  frequency
0      No   81776   0.803593
1  Steady   18345   0.180272
2      Up    1067   0.010485
3    Down     575   0.005650

repaglinide
--------------------------------------------------
   values  counts  frequency
0      No  100224   0.984877
1  Steady    1384   0.013600
2      Up     110   0.001081
3    Down      45   0.000442

nateglinide
--------------------------------------------------
   values  counts  frequency
0      No  101060   0.993092
1  Steady     668   0.006564
2      Up      24   0.000236
3    Down      11   0.000108

chlorpropamide
--------------------------------------------------
   values  counts  frequency
0      No  101677   0.999155
1  Steady      79   0.000776
2      Up       6   0.000059
3    Down       1   0.000010

glimepiride
--------------------------------------------------
   values  counts  frequency
0      No   96572   0.948989
1  Steady    4670   0.045891
2      Up   

In [36]:
dict_drug = {'No': 0, 
            'Down':1, 
            'Steady':2,
            'Up':3,
             'Ch':1,
             'Yes':1
            }
#replace drug text with numerical value

In [37]:
drug_df = df.loc[:, 'metformin':'diabetesMed']

In [38]:
for drug in drug_df:
    drug_df[drug] = drug_df[drug].map(dict_drug)


In [39]:
df.loc[:, 'metformin':'diabetesMed'] = drug_df

In [40]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,dataset,level1_diag1,level1_diag2,level1_diag3,med_dosage_change,number_of_medicine
0,7733208,3291489,1,0,65,?,1,1,7,2,?,Cardiology,51,3,11,0,0,0,786,530,250,4,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,train,4.0,7.0,0.0,0,0
1,152449578,84529188,1,1,45,?,1,2,7,3,HM,?,86,1,15,1,0,1,511,276,276,9,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,train,0.0,1.0,0.0,0,0
2,440311646,121372727,1,1,45,?,1,3,7,13,?,?,88,5,34,0,0,0,507,453,518,9,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,train,6.0,6.0,6.0,1,1
3,106684962,24066279,1,0,85,?,5,3,17,4,MC,Orthopedics-Reconstructive,18,2,17,3,2,0,820,285,428,9,0,-999,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,train,2.0,0.0,0.0,0,3
4,139779162,86645961,2,1,65,?,1,4,1,3,?,?,22,0,11,1,0,2,428,491,295,6,-999,-999,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,train,1.0,7.0,1.0,0,3
5,223277988,59559255,1,1,75,?,3,22,1,5,MC,?,68,1,25,0,1,0,822,585,427,9,-999,-999,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,train,8.0,4.0,0.0,0,2
6,127005696,72034335,1,1,85,?,3,1,1,4,?,Surgery-Thoracic,66,4,21,1,0,1,414,250,440,6,-999,-999,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,train,3.0,4.0,1.0,0,2
7,37320456,23560803,1,0,65,?,5,3,17,6,?,Family/GeneralPractice,17,0,9,0,1,0,331,250.01,294,3,0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,train,5.0,0.0,0.0,1,1
8,144033276,6382269,2,1,55,?,1,1,7,2,CP,InternalMedicine,43,6,13,0,0,0,414,593,250,6,-999,-999,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,train,2.0,1.0,0.0,0,2
9,166008048,89445645,1,1,75,?,2,1,1,2,?,InternalMedicine,52,3,13,1,0,0,410,414,250,9,-999,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,train,4.0,1.0,0.0,0,1


### Missing Values

In [41]:
df = df.replace('?', np.nan)
#saw in dataset missing value or unknown has '?', will replace with nan


In [42]:
df.isnull().sum().sort_values(ascending=False).head(10)
#list col with missing values


weight                98566
medical_specialty     49947
payer_code            40255
diag_3                 1423
diag_2                  358
diag_1                   21
number_of_medicine        0
number_outpatient         0
repaglinide               0
metformin                 0
dtype: int64

In [43]:
df = df.drop(['weight', 'payer_code', 'medical_specialty'], axis = 1)
#dropping columns with too many missing values

df = df.drop(['examide', 'citoglipton'], axis = 1)
#these 2 drugs were all 'no'


In [44]:
df = df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]
#11, 13, 14, 19, 20, 21 are patients who are expired so no re-admission possible


In [45]:
df.columns

Index(['Unnamed: 0', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'dataset', 'level1_diag1', 'level1_diag2', 'level1_diag3',
       'med_dosage_change', 'number_of_medicine'],
      dtype='object')

In [45]:
df = df.drop(['diag_1', 
              'diag_2', 
              'diag_3'], axis = 1)
#remove the columns that were engineered into other columns


In [46]:
df.shape

(99340, 48)

In [47]:
df['random'] = np.random.randint(-999, 999, df.shape[0])
df.random

0        782
1        609
2        560
3       -710
4        568
5       -779
6        872
7       -537
8        466
9        952
10        75
11       -87
12       579
13       -72
14      -179
15      -462
16      -306
17       983
18       457
19      -982
20      -961
22       389
23       408
24       197
25      -953
26       319
27       563
28       627
29       656
30       187
31       945
32      -318
33      -592
34      -670
35      -537
36       497
37       606
38       -45
39      -496
40       195
41      -176
42      -537
43       396
44      -887
45      -856
46       255
47         6
48      -255
49        -2
50      -754
51       146
52       491
53      -830
54      -290
55      -282
56       376
57       273
58       646
59       348
60       631
61      -797
62      -646
63       542
64      -522
65       159
66      -873
67       112
68       193
69      -259
70       643
71       -71
73      -519
74      -298
75      -166
76       459
77       235
78       632

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99340 entries, 0 to 101762
Data columns (total 50 columns):
Unnamed: 0                  99340 non-null int64
encounter_id                99340 non-null int64
patient_nbr                 99340 non-null int64
race                        99340 non-null int64
gender                      99340 non-null int64
age                         99340 non-null int64
admission_type_id           99340 non-null int64
discharge_disposition_id    99340 non-null int64
admission_source_id         99340 non-null int64
time_in_hospital            99340 non-null int64
num_lab_procedures          99340 non-null int64
num_procedures              99340 non-null int64
num_medications             99340 non-null int64
number_outpatient           99340 non-null int64
number_emergency            99340 non-null int64
number_inpatient            99340 non-null int64
number_diagnoses            99340 non-null int64
max_glu_serum               99340 non-null int64
A1Cresul

In [50]:
df.columns

Index(['Unnamed: 0', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'dataset', 'level1_diag1', 'level1_diag2', 'level1_diag3',
       'med_dosage_change', 'number_of_medicine', 'random'],
      dtype='object')

In [48]:
#convert to object dtype
df[['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'dataset', 'level1_diag1', 'level1_diag2', 'level1_diag3']] = df[['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'dataset', 'level1_diag1', 'level1_diag2', 'level1_diag3']].astype('object')


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99340 entries, 0 to 10176
Data columns (total 49 columns):
encounter_id                99340 non-null object
patient_nbr                 99340 non-null object
race                        99340 non-null object
gender                      99340 non-null object
age                         99340 non-null object
admission_type_id           99340 non-null object
discharge_disposition_id    99340 non-null object
admission_source_id         99340 non-null object
time_in_hospital            99340 non-null int64
num_lab_procedures          99340 non-null int64
num_procedures              99340 non-null int64
num_medications             99340 non-null int64
number_outpatient           99340 non-null int64
number_emergency            99340 non-null int64
number_inpatient            99340 non-null int64
number_diagnoses            99340 non-null int64
max_glu_serum               99340 non-null object
A1Cresult                   99340 non-null object

In [53]:
df.to_csv('./merged_data_preprocessed.csv')

In [54]:
df = pd.read_csv('./merged_data_preprocessed.csv')

In [50]:
df_train = df[df.dataset=='train']
df_train = df_train.drop(columns='dataset')
df_train.sample(5)

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,level1_diag1,level1_diag2,level1_diag3,med_dosage_change,number_of_medicine,random
62992,34531164,1176696,1,1,65,1,1,7,2,54,0,8,0,0,0,8,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,1,7,0,0,392
30104,19782750,78990885,-999,0,75,6,1,17,2,58,2,5,0,0,0,7,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,3,0,3,0,1,447
44193,71584620,16740945,1,0,85,2,1,4,2,37,2,16,0,0,0,9,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,5,1,1,0,1,405
42753,87653214,1048914,2,0,75,1,4,6,1,49,0,11,0,0,1,5,-999,-999,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,1,0,7,0,0,0,2,231
82916,261872754,104015889,1,0,45,2,1,1,4,30,6,26,0,1,0,8,-999,-999,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,6,6,6,0,2,431


In [51]:
df_test = df[df.dataset=='test']
df_test = df_test.drop(columns='dataset')
df_test.sample(5)

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,level1_diag1,level1_diag2,level1_diag3,med_dosage_change,number_of_medicine,random
4375,98136504,1892088,1,0,95,1,1,6,5,50,0,9,0,0,1,5,-999,-999,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,1,2,0,1,464
8404,54345006,6040503,1,1,65,3,1,1,1,40,5,11,1,0,1,6,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,0,-664
2601,333074780,38544030,-999,1,75,1,3,7,3,54,0,11,0,0,1,9,-999,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0,0,7,5,1,1,-422
7772,69214074,28553517,1,0,55,5,3,17,4,26,0,18,0,0,0,9,0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,7,1,1,1,665
6018,275418096,38751660,1,0,75,3,1,4,5,43,1,24,0,0,0,9,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0,2,1,7,1,1,-669


In [54]:
df_test.to_csv('./df_test_cleaned.csv', index=False)
df_train.to_csv('./df_train_cleaned.csv', index=False)

In [55]:
df_test.sample(5)

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,level1_diag1,level1_diag2,level1_diag3,med_dosage_change,number_of_medicine,random
9881,146810268,84747474,1,1,65,3,1,1,2,28,3,13,0,0,1,9,-999,-999,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,8,8,8,0,1,930
6894,27686652,7362756,4,0,55,2,1,4,4,37,0,17,0,0,0,5,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,1,0,4,0,1,-238
773,135537894,107802315,1,1,65,2,1,7,1,28,0,11,0,0,0,9,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,2,0,2,1,1,-606
8193,140340852,41508126,1,0,55,1,1,7,1,3,0,11,0,0,0,6,-999,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,879
3197,158782212,81701730,2,0,45,2,1,1,11,45,1,22,0,0,1,7,-999,-999,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,0,0,0,2,-362
